In [5]:
import os
from os.path import join
import json
import re

import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kpaks\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [70]:
documents_path = join('..', 'data', 'captions')
documents = os.listdir(documents_path)

metadata = pd.read_csv(join('..', 'data', 'metadata.csv'))

dfs = []
for document in documents:
    df = pd.read_csv(join(documents_path, document))
    df['title'] = document[:-4]
    df = df.reset_index()
    df = df.set_index(['title', 'name'])
    dfs.append(df)

In [71]:
findf = pd.concat(dfs)
findf.head()

index  \
title                                              name                                                       
100K_Thank_You_The_Story_and_Mission_of_Healthy... None                                                   0   
10_Minute_Daily_Meditation                         None                                                   0   
12_Rules_For_Life                                  - RULE 9: Success is dependent on your raid guild      0   
                                                   - RULE 10: Any build is viable                         1   
                                                   - RULE 11: Exploit the meta                            2   

                                                                                                       time  \
title                                              name                                                       
100K_Thank_You_The_Story_and_Mission_of_Healthy... None                                                 0.0   
10_Minute_Daily_Meditation                         None                                                 0.0   
12_Rules_For_Life                                  - RULE 9: Success is dependent on your raid guild  678.0   
                                                   - RULE 10: Any build is viable                     762.0   
                                                   - RULE 11: Exploit the meta                        848.0   

                                                                                                                                               captions  
title                                              name                                                                                                  
100K_Thank_You_The_Story_and_Mission_of_Healthy... None                                               so we have hid miraculously like eight weeks a...  
10_Minute_Daily_Meditation                         None                                               so you guys want to learn a little bit of medi...  
12_Rules_For_Life                                  - RULE 9: Success is dependent on your raid guild  next thing success is dependent on your raid g...  
                                                   - RULE 10: Any build is viable                     any build is viable don't compare so this is w...  
                                                   - RULE 11: Exploit the meta                        rule number 11 exploit the meta so just like v...

In [72]:
title_map = findf.index.values

In [74]:
texts = findf['captions'].values

In [32]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

In [115]:
def dot_score(a, b):
    if len(a.shape) == 1:
        a = np.expand_dims(a, 0)

    if len(b.shape) == 1:
        b = np.expand_dims(b, 0)

    return np.matmul(a, b.T)

In [116]:
dot_score(query_enc, query_enc)

array([[1.0000001]], dtype=float32)

In [33]:
enc_txts = model.encode(texts)

In [79]:
# np.savetxt('enc_txts.csv', enc_txts, delimiter=',')

In [80]:
query = 'I cannot communicate with my parents.'

In [124]:
# all at once
query_enc = model.encode(query)
scores = util.dot_score(query_enc, enc_txts)
scores = scores.numpy()[0]
# scores = dot_score(query_enc, enc_txts)
# scores = scores[0]
ordered = pd.factorize(scores, sort=True)[0]
df = pd.DataFrame( (scores, ordered), index=['score', 'order'] ).T.sort_values(by='order', ascending=False)
idx = df.index.values
relevant_titles = title_map[idx]
scores_sorted = scores[idx]

for title, score in zip(relevant_titles[:15], scores_sorted[:15]):
    print(title, score)

('Self_Love_with_Sweet_Anita_Dr_K_Interviews', 'What did the parents model for you?') 0.44051635
('What_To_Do_If_Your_Parents_Don’t_Accept_You', '- Summary') 0.4383915
('Journey_to_Pro-Gaming_Confidence_Family_ft_Broxah', 'Parents Support and Divorce, Brother Story') 0.42088735
('How_Parents_Standards_Still_Affect_Us_ft_Natsumiii', 'Parents Aren’t Perfect') 0.41967028
('What_To_Do_If_Your_Parents_Don’t_Accept_You', '- Overview') 0.4195766
('Dr_Ks_Advice_to_talking_with_Parents', 'None') 0.41629452
('What_To_Do_If_Your_Parents_Don’t_Accept_You', '- Reddit Post') 0.40792805
('Helping_a_Father_and_Son_Reconnect_Dr_K_Interviews', 'What to change the family dynamic') 0.4045156
('Helping_a_Father_and_Son_Reconnect_Dr_K_Interviews', 'Talking with Mom vs Dad') 0.39870635
('Why_You_Cant_Really_Fix_Your_Siblings_Mental_Health', '- Concrete communication') 0.39537394
('Why_You_Wasted_The_Last_5_Years_Of_Your_Life', '- What to remember') 0.39505723
('Keeping_a_Family_Together_after_Losing_a_Parent

# BERT

In [4]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

query_embedding = model.encode('How big is London')
passage_embedding = model.encode(['London has 9,787,426 inhabitants at the 2011 census',
                                  'London is known for its finacial district',
                                  'london is known for its finacial district'])

In [5]:
print("Similarity:", util.dot_score(query_embedding, passage_embedding))

Similarity: tensor([[0.5472, 0.6330, 0.6330]])


In [6]:
sub_titles = np.array(titles)

In [7]:
enc_txts = model.encode(texts)

In [17]:
enc_txts.shape

(473, 384)

In [18]:
enc_df = pd.DataFrame(enc_txts)
enc_df['title'] = titles
enc_df.to_csv('..//encoded_titles.csv', index=False)

In [15]:
import pickle

with open('..//model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [148]:
que = 'I have problems with my self esteem.'

In [149]:
# all at once
query = model.encode(que)
scores = util.dot_score(query, enc_txts)
scores = scores.numpy()[0]
ordered = pd.factorize(scores, sort=True)[0]
df = pd.DataFrame( (scores, ordered), index=['score', 'order'] ).T.sort_values(by='order', ascending=False)
idx = df.index.values
relevant_titles = sub_titles[idx]
scores_sorted = scores[idx]

for title, score in zip(relevant_titles[:5], scores_sorted[:5]):
    print(title, score)

How_To_Build_Your_Self-Esteem 0.61193335
Can_You_Really_Change_Who_You_Are 0.518962
Are_You_More_Than_Your_Sex_Life__Overcoming_Jealousy 0.49866542
Psychiatrist_on_Burnt_Out_Students 0.4970919
I_Am_Too_Soft_To_Be_A_Man 0.47713417


In [151]:
[x for x in titles if 'beating_yourself' in x.lower()]

['Dr._K_Meets_the_Raid_Boss__Self-Esteem_&_Beating_Yourself_Up']

In [116]:
query = model.encode(que)

In [162]:
scores = util.dot_score(query, enc_txts)
scores = scores.numpy()[0]
ordered = pd.factorize(scores, sort=True)[0]

In [163]:
df = pd.DataFrame( (scores, ordered), index=['score', 'order'] ).T.sort_values(by='order', ascending=False)
idx = df.index.values

In [164]:
sub_titles[idx]

array(['How_To_Build_Your_Self-Esteem',
       'Can_You_Really_Change_Who_You_Are',
       'Are_You_More_Than_Your_Sex_Life__Overcoming_Jealousy',
       'Psychiatrist_on_Burnt_Out_Students', 'I_Am_Too_Soft_To_Be_A_Man',
       'Why_Do_I_Get_So_Mad_When_Others_Are_Wrong',
       'Why_Shame_is_the_Raid_Boss_of_Emotions',
       'Why_You_Are_Always_Unhappy', 'Nontraditional_Narcissists',
       'Why_Your_Knowledge_Is_Not_Your_Own',
       'I_Am_Too_Ugly_for_a_Girlfriend',
       'Therapist_Talks_Why_Self-Pity_Can_Feel_So_Good',
       'I_Used_To_Be_Successful...',
       'Psychiatrist_on_solving_I_lack_human_connection',
       'Psychiatrist_reacts_to_I_have_too_much_self-awareness',
       'You_Dont_Want_To_Be_Helped', "You_Don't_Want_To_Be_Helped",
       'Why_You_Feel_Like_Everyone_Else_Is_Stupid',
       'Should_I_Make_What_I_Like_Doing_My_Job_Passion__Purpose',
       'Why_You_Cant_Leave_Your_House', "Why_You_Can't_Leave_Your_House",
       'Why_You_Dont_Feel_Accepted_ft._Bobby_Scar

In [161]:
sub_titles[idx]

array(['How_To_Build_Your_Self-Esteem',
       'Can_You_Really_Change_Who_You_Are',
       'Are_You_More_Than_Your_Sex_Life__Overcoming_Jealousy',
       'Psychiatrist_on_Burnt_Out_Students', 'I_Am_Too_Soft_To_Be_A_Man',
       'Why_Do_I_Get_So_Mad_When_Others_Are_Wrong',
       'Why_Shame_is_the_Raid_Boss_of_Emotions',
       'Why_You_Are_Always_Unhappy', 'Nontraditional_Narcissists',
       'Why_Your_Knowledge_Is_Not_Your_Own',
       'I_Am_Too_Ugly_for_a_Girlfriend',
       'Therapist_Talks_Why_Self-Pity_Can_Feel_So_Good',
       'I_Used_To_Be_Successful...',
       'Psychiatrist_on_solving_I_lack_human_connection',
       'Psychiatrist_reacts_to_I_have_too_much_self-awareness',
       'You_Dont_Want_To_Be_Helped', "You_Don't_Want_To_Be_Helped",
       'Why_You_Feel_Like_Everyone_Else_Is_Stupid',
       'Should_I_Make_What_I_Like_Doing_My_Job_Passion__Purpose',
       'Why_You_Cant_Leave_Your_House', "Why_You_Can't_Leave_Your_House",
       'Why_You_Dont_Feel_Accepted_ft._Bobby_Scar

In [155]:
np.where(sub_titles == 'Dr._K_Meets_the_Raid_Boss__Self-Esteem_&_Beating_Yourself_Up')

(array([66], dtype=int64),)

In [ ]:
s